In [1]:
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np
import cv2
import mediapipe as mp
import torch
import pandas as pd
from scipy.fft import fft, ifft


In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")
torch.cuda.empty_cache()

Using cuda device


In [3]:
def draw_landmarks_on_image(rgb_image, detection_result):
  pose_landmarks_list = detection_result.pose_landmarks
  annotated_image = np.copy(rgb_image)

  # Loop through the detected poses to visualize.
  for idx in range(len(pose_landmarks_list)):
    pose_landmarks = pose_landmarks_list[idx]

    # Draw the pose landmarks.
    pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    pose_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in pose_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      pose_landmarks_proto,
      solutions.pose.POSE_CONNECTIONS,
      solutions.drawing_styles.get_default_pose_landmarks_style())
  return annotated_image

In [4]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

In [5]:

def getFiturLandmark(img):
    err = None
    detection_result = []
    try:
        
        base_options = python.BaseOptions(model_asset_path='pose_landmarker_heavy.task')
        options = vision.PoseLandmarkerOptions(
        base_options=base_options,
        output_segmentation_masks=True)
        detector = vision.PoseLandmarker.create_from_options(options)
        
        detection_result = detector.detect(img)

        err = None
    
    except:
        err = 1
        detection_result = []
    
    return detection_result, err



    
    

In [6]:
# ainput = mp.Image.create_from_file("../../dataset/extractFrame/above/00431/121.jpg")
# aaa, aer = getFiturLandmark(ainput)
# aaalandmarklist = aaa.pose_landmarks
# # _dfpose = pd.DataFrame(aaalandmarklist[0])
# print(len(aaalandmarklist))

In [7]:
# concate 
def flatten(data):
    a = np.ravel(data)
    return a

In [8]:
# feed to the dataset huhuyy

import os
import pandas as pd
from scipy.fft import fft, ifft


def load_feature_from_folder(folder):
  img = []
  data = []
  for folderName in os.listdir(folder):
    print(folderName)
    folder1 = os.path.join(folder, folderName)
    for foll in os.listdir(folder1):
      print(foll)
      folder2 = os.path.join(folder1, foll)
      cnt = 0
      for fileInside in os.listdir(folder2):
        pathLengkap = os.path.join(folder2, fileInside)
        namaClass = folderName
        # mp_image = mp.Image.create_from_file(pathLengkap)
        # result, err = getFiturLandmark(mp_image)
        
        
        if(True):
          try:
            # pose_landmarks_list = result.pose_landmarks
            # dfpose = pd.DataFrame(pose_landmarks_list[0])
            # print(cnt)
            # print(pose_landmarks_list[0])
            # hasil1 = flatten(dfpose)
            # hasilfft = np.fft.fft(hasil1)
            masuk_data = {
              'class': namaClass,
              'namaFile': foll,
              'urutan': cnt,
              # 'feature_lengkap': dfpose,
              # 'triangle_feature': hasil1,
              # 'fourier_feature': hasilfft
            }
            cnt = cnt + 1
            data.append(masuk_data)
            # print(data)
          except:
            print(pathLengkap)
        else:
          continue
  dataFrame = pd.DataFrame(data)
  dataFrame.to_pickle("data_keypoint_fitur_lengkap_pickle")
         
path_dataset = "../../dataset/extractFrame"
load_feature_from_folder(path_dataset)

above
00430
00431
00433
00435
65004
accomplish
00663
00664
00666
00668
65010
adopt
01157
01158
01159
01160
01162
65022
advantage
01244
01245
01252
65025
alphabet
02145
02149
02151
65058
anniversary
02609
02610
02611
02614
02616
another
02697
02698
02699
02701
02706
appropriate
03101
03102
03103
03105
03108
arrogant
03448
03449
03450
03451
03454
03457
artist
03515
03516
03517
03519
03522
background
04580
04581
04582
04590
65122
bee
05680
05681
05682
05685
05688
65165
behavior
05792
05793
05794
05796
05798
bell
05873
05874
05875
05877
05878
05881
birth
06372
06373
06375
06379
65189
65190
bless
06625
06626
06628
06629
06631
65206
blood
06733
06734
06736
06738
65212
boots
07163
07173
65226
brave
07575
07576
07578
07580
65251
breathe
07676
07677
07678
07680
07682
65256
bright
07779
07780
07781
07782
07787
07791
bull
08166
08167
08168
08170
08174
butterfly
08442
08443
08445
08447
65280
card
09126
09127
09128
09130
09131
09137
09138
65304
category
09488
09489
09490
09493
09495
catholic
09518


In [9]:
# buat listfile csv

def createListFile(_folder):
    _data = []
    for _folderName in os.listdir(_folder):
        print(_folderName)
        _folder1 = os.path.join(_folder, _folderName)
        for _foll in os.listdir(_folder1):
            print(_foll)
            _folder2 = os.path.join(_folder1, _foll)
            cnt = 0
            for _fileInside in os.listdir(_folder2):
                _pathLengkap = os.path.join(_folder2, _fileInside)
                _namaClass = os.path.join(_folderName,_foll, _fileInside)
                print(_namaClass)
                _data.append(_namaClass)
    
    _dtframe = pd.DataFrame(_data)
    _dtframe.to_pickle("list_file")

path_dataset = "../../dataset/extractFrame"
# createListFile(path_dataset)

In [10]:
# create dataset class
import os
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from sklearn.cluster import KMeans


class dataClass(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.file = pd.read_pickle("list_file")
        self.root_dir = root_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.file)
    
    def __getitem__(self, index):
        _path_depan = self.root_dir
        _path_belakang = self.file.iloc[index,0]
        _path_belakang = _path_belakang.replace(os.sep, '/')
        _path_lengkap = os.path.join(_path_depan, _path_belakang)
        _image = cv2.imread(_path_lengkap)
        _mpImage = mp.Image.create_from_file(_path_lengkap)
        _result, _err = getFiturLandmark(_mpImage)
        _pose_landmarks_list = _result.pose_landmarks
        if(len(_pose_landmarks_list)>0):
            
            _dfpose = pd.DataFrame(_pose_landmarks_list[0])
            _dfposeFlatten = flatten(_dfpose)
            _datareturn = {'path': _path_lengkap,
                       'image': _image,
                       'fitur_keypoint_lengkap': _result,
                       'fitur_lengkap_flatten': _dfposeFlatten,
                       }
        else:
            return self.__getitem__(index=index+1)
        return _datareturn

dataaset_torch = dataClass(csv_file="list_file", root_dir="../../dataset/extractFrame/")   


In [12]:
data_lengkap = pd.read_pickle("../../dataset/data_keypoint_fitur_lengkap_pickle")
# data_lengkap.iloc[0]["triangle_feature"]
a = data_lengkap
a


,class,namaFile,urutan,feature_lengkap,triangle_feature,fourier_feature
0,above,00430,0,x y z visibility ...,"[0.506881058216095, 0.2079756259918213, -0.689...","[(78.45452938933158+0j), (-4.464086183401866+3..."
1,above,00430,1,x y z visibility ...,"[0.5017372369766235, 0.2056875228881836, -0.77...","[(76.56170868576737+0j), (-3.980746325549271+3..."
2,above,00430,2,x y z visibility ...,"[0.5007808804512024, 0.20154517889022827, -0.5...","[(76.98094305739505+0j), (-3.169821885252973+0..."
3,above,00430,3,x y z visibility ...,"[0.49879002571105957, 0.19752609729766846, -0....","[(75.20907570922282+0j), (-2.6053617781116314+..."
4,above,00430,4,x y z visibility ...,"[0.49864041805267334, 0.18816208839416504, -0....","[(75.98558826980297+0j), (-2.356283168908329+1..."
...,...,...,...,...,...,...
38725,yourself,64451,24,x y z visibility ...,"[0.5141703486442566, 0.2702227234840393, -0.78...","[(76.11904963408597+0j), (-1.9968639410799702+..."
38726,yourself,64451,25,x y z visibility ...,"[0.5149209499359131, 0.2688581943511963, -0.79...","[(75.3994476548396+0j), (-1.6359811104954964+2..."
38727,yourself,64451,26,x y z visibility ...,"[0.5143306851387024, 0.2708406448364258, -0.78...","[(75.75465970579535+0j), (-1.9866186968249133+..."
38728,yourself,64451,27,x y z visibility ...,"[0.5157540440559387, 0.2723342180252075, -0.80...","[(75.77654468570836+0j), (-1.8139997132680858+..."


In [ ]:
# define LSTM model

